In [1]:
# load china AS names
import json
fp = open("info/AS_info.json")
data = json.load(fp)
fp.close()
name_cn = [i for i in data if data[i]['zone'] == "CHN"]
name_jak = [i for i in data if data[i]['zone'] == "JAK"]

# Read data

In [2]:
import numpy as np
import networkx as nx
# import nxmetis as nxm
from matplotlib import pyplot as plt
import scipy
# import graphviz as gv
import igraph as ig
import gurobipy as gp
from gurobipy import GRB

np.random.seed(5)

pngdir = 'pngs/'
txtdir = 'txts/'

g = nx.read_adjlist(txtdir+"adj_without_loop.txt")

# a = dict(g.degree())

# nums = [1500,2000,5000]

# # choosing nodes with high degrees
# rankg = {i: g.subgraph([k[0] for k in sorted(a.items(),key=lambda x:x[1],reverse=True)[:i]]) for i in nums}


# # choosing nodes randomly with different probability

# def normalized(a):
#     return a / a.sum()

# pp = {'sqrt': normalized(np.array([np.sqrt(i[1]) for i in a.items()])),
#       'log' : normalized(np.array([np.log(i[1]) for i in a.items()])),
#       'lin' : normalized(np.array([i[1] for i in a.items()])),
#                         }

# sqrtg = {i:g.subgraph(np.random.choice(np.array(list(g.nodes)),i,p=pp['sqrt'],replace=False)) for i in nums}
# ling = {i:g.subgraph(np.random.choice(np.array(list(g.nodes)),i,p=pp['lin'],replace=False)) for i in nums}
# logg = {i:g.subgraph(np.random.choice(np.array(list(g.nodes)),i,p=pp['log'],replace=False)) for i in nums}
# randg = {i:g.subgraph(np.random.choice(np.array(list(g.nodes)),i,replace=False)) for i in nums}

# choosing CHINA ASes
cn = g.subgraph(name_cn)
cnjak = g.subgraph(name_cn+name_jak)
# choosing CHINA and its neighbors
# nx.write_adjlist(cn,txtdir+'CHN.txt')

# G = ig.Graph.from_networkx(cn)

# sqrtig = {i:ig.Graph.from_networkx(sqrtg[i]) for i in [2000]}
# linig = {i:ig.Graph.from_networkx(ling[i]) for i in [2000]}
# logig = {i:ig.Graph.from_networkx(logg[i]) for i in [2000]}
# randig = {i:ig.Graph.from_networkx(randg[i]) for i in [2000]}

NameError: name 'name_cn' is not defined

In [3]:
# a=list(g.degree())
# a.sort(key=lambda x:x[1],reverse=True)
# # len([i for i in a if i[1] > 1000])
# a
# cn.nodes
# a.sort(key=lambda x:x[1],reverse=True)

largest_cc = cnjak.subgraph(max(nx.connected_components(cnjak), key=len))
print(len(cnjak.nodes),len(largest_cc.nodes))

4420 4126


In [99]:
len(cnjak.edges)

10746

In [29]:
gnp_5w_0_0015 = nx.fast_gnp_random_graph(50000,0.0015,seed=5)

sf_5w_7 = nx.barabasi_albert_graph(50000,8,seed=5)

as_like = nx.random_internet_as_graph(10000,seed=5)

tree_like_5w = nx.random_tree(50000)

for i in tree_like_5w.edges:
    tree_like_5w.edges[i]['weight'] = np.random.randint(1,11)

tree_like_5w.edges

G = as_like
index = dict(zip(G, list(range(len(G)))))
xadj = [0]
adjncy = []
for u in G:
    adjncy.extend(index[v] for v in G[u])
    xadj.append(len(adjncy))

adjncy

import metis

metisg = metis.networkx_to_metis(as_like)
metisg

In [72]:
#ig_gnp = ig.Graph.from_networkx(gnp_5w_0_0015)
#ig_sf = ig.Graph.from_networkx(sf_5w_7)
ig_as = ig.Graph.from_networkx(as_like)
#ig_tree = ig.Graph.from_networkx(tree_like_5w)

In [101]:
ig_gnp_5k = ig.Graph.Erdos_Renyi(5000,m=20000)
ig_gnp_1w = ig.Graph.Erdos_Renyi(10000,m=40000)
ig_gnp_5w = ig.Graph.Erdos_Renyi(50000,m=200000)
ig_powerl_5k = ig.Graph.Static_Power_Law(5000,20000,2.5)
ig_powerl_1w = ig.Graph.Static_Power_Law(10000,40000,2.5)
ig_powerl_5w = ig.Graph.Static_Power_Law(50000,200000,2.5)


#ig_tree = ig.Graph.Tree(50000)

In [109]:
ig_gnp_5w = ig.Graph.Erdos_Renyi(50000,m=200000)

In [ ]:
ig_all_as = ig.Graph.from_networkx(g)

# save to pickle and txt

In [110]:
import pickle
pickledir = "middleResults/"
ig_cnakj = ig.Graph.from_networkx(largest_cc)
experiment_graphs = {'5k': {'as': ig_cnakj, 'gnp':ig_gnp_5k,'pl':ig_powerl_5k},
                     '1w': {'as':ig_as, 'gnp':ig_gnp_1w, 'pl': ig_powerl_1w},
                     '5w': {'gnp':ig_gnp_5w,'pl':ig_powerl_5w, 'as':ig_all_as}
                    }
for size in experiment_graphs:
    for g in experiment_graphs[size]:
        for i in experiment_graphs[size][g].es:
            i['weight'] = 1

In [112]:
# dump as pickle
#pickle.dump(largest_cc,open(pickledir+"cc_cnajk","wb"))
pickle.dump(experiment_graphs,open(pickledir+"experiment_graphs","wb"))

In [113]:
# write as txt 
for size in experiment_graphs:
    for graphname in experiment_graphs[size]:
        graph = experiment_graphs[size][graphname]
        ll = graph.get_adjlist()
        with open(pickledir+graphname+'-'+size+'.txt','w') as f:
            f.write("{} {}\n".format(len(graph.vs),len(graph.es)))
            for i in range(len(ll)):
                for j in ll[i]:
                    f.write("{} ".format(j+1))     # id starts from 1
                f.write("\n")

        if graphname == "as" and size == "5k":
        # write id-name map
            nowg = graph
            node_list = list(nowg.vs['_nx_name'])
            node_map_id2name = {i:node_list[i] for i in range(len(node_list))}
            node_map_name2id = {node_list[i]:i for i in range(len(node_list))}
            import pickle
            from datetime import date
            td = str(date.today())
            pickle.dump( node_map_id2name , open(pickledir+graphname+size+"-node_map_id2name-"+td,"wb"))
            pickle.dump( node_map_name2id , open(pickledir+graphname+size+"-node_map_name2id-"+td,"wb"))

# 2500 data

# Fat tree

In [6]:
k=20
(k**3)/4 + (k**2) + (k**2)/4

2500.0

In [8]:
import networkx as nx

fatt = {}
for k in [20]:

    g = ig.Graph()
#     k = 24
    step = int(k/2)
    # k=24, 4176
    # -------k=26, 5239
    # k=32, 9472
    #--------- k=34, 11271
    # k=56, 47824
    # ---------k=58, 52983

    # (k**3)/4+ (k**2)/4 + (k*k)

    g.add_vertices(['core'] * int((k**2)/4))
    g.add_vertices(['aggr'] * int((k**2)/2))
    g.add_vertices(['edge'] * int((k**2)/2))
    g.add_vertices(['host'] * int((k**3)/4))

    cores = g.vs.select(name='core')
    core = [i.index for i in cores]
    aggrs = g.vs.select(name='aggr')
    aggr = [i.index for i in aggrs]
    edges = g.vs.select(name='edge')
    edge = [i.index for i in edges]
    hosts = g.vs.select(name='host')
    host = [i.index for i in hosts]

    for i in range(len(edge)):
        for j in range(step):
            g.add_edges([(edge[i],host[i*step+j])])

    for x in range(k):
        for i in range(step):
            for j in range(step):
                g.add_edges([(aggr[x*step+i], edge[x*step+j])])

    for x in range(k):
        for i in range(step):
            for j in range(step):
                g.add_edges([(aggr[x*step+i], core[i*step+j])])
                
    fatt[k] = g



In [11]:
len(fatt[20].es)

6000

In [230]:
for j in fatt:
    for i in fatt[j].es:
        i['weight'] = 1

In [231]:
pickle.dump(fatt,open(pickledir+"fatt","wb"))

In [233]:
for size in fatt:
    graph = fatt[size]
    ll = graph.get_adjlist()
    with open(pickledir+'fattree-k='+str(size)+'.txt','w') as f:
        f.write("{} {}\n".format(len(graph.vs),len(graph.es)))
        for i in range(len(ll)):
            for j in ll[i]:
                f.write("{} ".format(j+1))     # id starts from 1
            f.write("\n")



In [12]:
gnp_2_5k = ig.Graph.Erdos_Renyi(2500,m=10000)
powerl_2_5k = ig.Graph.Static_Power_Law(2500,10000,2.5)
nx_as_2_5k = nx.random_internet_as_graph(2500,seed=5)
as_2_5k = ig.Graph.from_networkx(nx_as_2_5k)
ft_2_5k = fatt[20]

import pickle
pickledir = "middleResults/"
exp_2500 = {'gnp':gnp_2_5k, 'pl': powerl_2_5k, 'as': as_2_5k, 'ft': ft_2_5k}
for g in exp_2500:
    for i in exp_2500[g].es:
        i['weight'] = 1

# dump as pickle
#pickle.dump(largest_cc,open(pickledir+"cc_cnajk","wb"))
pickle.dump(exp_2500,open(pickledir+"exp_2500","wb"))

# write as txt 
for graphname in exp_2500:
    graph = exp_2500[graphname]
    ll = graph.get_adjlist()
    with open(pickledir+graphname+'-'+'2.5k.txt','w') as f:
        f.write("{} {}\n".format(len(graph.vs),len(graph.es)))
        for i in range(len(ll)):
            for j in ll[i]:
                f.write("{} ".format(j+1))     # id starts from 1
            f.write("\n")

